In [ ]:
import sys
from pathlib import Path

# add directory with ribasim-repository
sys.path.append("..\\..\\ribasim\\python\\ribasim")
import ribasim

# add directory with ribasim_lumping-repository
sys.path.append("..")
import ribasim_lumping

import pandas as pd
import geopandas as gpd

from numba.core.errors import NumbaDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)

import networkx as nx

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Define base_dir, results_dir and network name
base_dir = Path("..\\..\\ribasim_lumping_data\\")
dhydro_dir = Path(base_dir, "d-hydro\\tki_hooge_raam\\results\\")
results_dir = Path(base_dir, "results")
network_name = "tki_hooge_raam_all"

In [ ]:
# Load areas (discharge units: afwaterende eenheden)
areas_file_path = Path(base_dir, "afw_eenheden", "afwateringseenheden2023_v1_HoogeRaam.shp")
areas_gdf = gpd.read_file(areas_file_path)
areas_gdf = areas_gdf[['CODE', 'geometry']]

In [ ]:
# Create networkanalysis
network = ribasim_lumping.create_ribasim_lumping_network(
    name=network_name, 
    dhydro_dir=dhydro_dir,
    results_dir=results_dir,
    areas_gdf=areas_gdf,
)
# network.export_to_geopackage()

In [ ]:
# Select simulation sets and extract all data using xugrid/ugrid
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=dhydro_dir,
    simulations_names=["0_01", "0_02", "0_05", "0_075", "0_1", "0_2", "0_5", "0_75", "1_0", "1_5", "2_0"],
    simulations_ts=[-1],
    simulations_output_dir="dflowfm\\output\\",
);

In [ ]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.get_network_data()
# network.export_to_geopackage()

In [ ]:
# Define locations where the network should be split into Ribasim basins:
network.add_split_nodes(
    weirs=True,
    pumps=False,
    uniweirs=False,
    edges=False,
    structures_ids_to_include=[], 
    structures_ids_to_exclude=['105BBW'], 
    node_ids_to_include=[],
    node_ids_to_exclude=[],
);

In [ ]:
# Create basins (gdf) based on nodes, edges, split_node_ids and areas
network.create_basins_and_connections_based_on_split_nodes();
# Export to geopackage
network.export_to_geopackage()

In [ ]:
# specify translation split_nodes to ribasim-nodes per type
split_node_type_conversion = {
    'weir': 'TabulatedRatingCurve', 
    'uniweir': 'TabulatedRatingCurve' ,
    'pump': 'Pump', 
    'culvert':'TabulatedRatingCurve', 
    'edge': 'ManningResistance',
    'orifice' : 'TabulatedRatingCurve'
}
# specify translation for specific split_nodes to type of ribasim-node
split_node_id_conversion = {}

In [ ]:
# Generate Ribasim model and export to geopackage
ribasim_model = network.generate_ribasim_model(
    split_node_type_conversion=split_node_type_conversion, 
    split_node_id_conversion=split_node_id_conversion
)
ribasim_model.write(Path(results_dir, network.name))